In [ ]:
import kplr 
client = kplr.API()
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import astropy.stats
import astropy
from astropy.stats import LombScargle
import math
import pylab
import scipy.ndimage

In [ ]:
starsample = client.star(6780873)
curves=starsample.get_light_curves(short_cadence=False, long_cadence=True, fetch=True, clobber=False)

In [ ]:
short_curves, long_curves = [], []

In [ ]:
for lc in curves:
    if "llc" in lc.filename:
        long_curves.append(lc)
    else:
        short_curves.append(lc)

In [ ]:
time, flux, flerr, quarter = np.array([]), np.array([]), np.array([]), np.array([])
for qq,lc in enumerate(long_curves): #same as KIC1718594 but changed short to long curves?
    print(lc.filename)
    with lc.open() as f:
        thisdata = f[1].data
        thistime = thisdata["time"]
        thisflux = thisdata["pdcsap_flux"]
        medianflux = np.nanmedian(thisflux)
        bad = np.logical_not(np.isfinite(thisflux))
        thisflux = thisflux / medianflux
        thisflux[bad] = 1.
        thisflerr = thisdata["pdcsap_flux_err"] / medianflux
        thisflerr[bad] = 1000.
        thisquarter = np.zeros_like(thistime) + qq
        time = np.concatenate((time, thistime))
        flux = np.concatenate((flux, thisflux))
        flerr = np.concatenate((flerr, thisflerr))
        quarter = np.concatenate((quarter, thisquarter))
quarter = np.round(quarter).astype(int)
print(time.shape, flux.shape, flerr.shape)
good = np.isfinite(time)
time = time[good]
flux = flux[good]
flerr = flerr[good]
quarter = quarter[good]
print(time.shape, flux.shape, flerr.shape)

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(time,flux,marker=".",alpha=0.1, c=quarter) #not quarters like before but looks cool
plt.xlabel("time",fontsize=20)
plt.ylabel("flux",fontsize=20)

In [ ]:
for foo in (time, flux, flerr, quarter):
    if not np.all(np.isfinite(foo)):
        bad = np.logical_not(np.isfinite(foo))
        print(np.sum(bad))

In [ ]:
I = (quarter < 100) #error unless at 0?
q,y= LombScargle(time[I],flux[I]).autopower()
#9.15 day period according to SJM

In [ ]:
#units: horizontal axis - frequency (cycles per day)
#verticle axis - lombscargle
plt.plot(q,y)
plt.xlabel("Frequency(cycles/day)")
plt.ylabel("LombScargle")
plt.title("All") #looks very regular

In [ ]:
plt.title("Zoomed-In Section")
plt.plot(q,y)
plt.xlabel("Frequency(cycles/day)")
plt.ylabel("LombScargle")
plt.xlim(13.4358,13.437)
plt.grid()

In [ ]:
plt.title("Zoomed-In Section")
plt.plot(q,y)
plt.xlabel("Frequency(cycles/day)")
plt.ylabel("LombScargle")
plt.xlim(14.18763,14.18764)
plt.ylim(.08,.1)
plt.grid()
plt.axvline(14.187637)

In [ ]:
plt.title("Zoomed-In Section 13.44")
plt.plot(q,y)
plt.xlabel("Frequency(cycles/day)")
plt.ylabel("LombScargle")
plt.xlim(13.434, 13.439)

In [ ]:
f0=14.1877
bandwidth=.2
fA=f0-bandwidth
fB=f0+bandwidth

In [ ]:
Ua=np.exp(2. * np.pi * 1j * fA * time)
Ub=np.exp(2. * np.pi * 1j * fB * time)
print(Ua.shape, Ub.shape)

In [ ]:
flux_zeromean = flux - np.mean(flux)
rA=Ua*flux_zeromean
rB=Ub*flux_zeromean
rS=Ua*Ub
print(rA.shape, rB.shape)
print(np.mean(flux))

In [ ]:
#gaussian smooth?
sAreal=scipy.ndimage.filters.gaussian_filter1d(np.real(rA),40) #magic 30
sBreal=scipy.ndimage.filters.gaussian_filter1d(np.real(rB),40)
sAimag=scipy.ndimage.filters.gaussian_filter1d(np.imag(rA),40)
sBimag=scipy.ndimage.filters.gaussian_filter1d(np.imag(rB),40)

In [ ]:
sA = (sAreal ** 2 + sAimag ** 2)
sB = (sBreal ** 2 + sBimag ** 2)
output=(sA-sB)/(sA+sB)

In [ ]:
plt.plot(time, output)
plt.xlabel("Time")
plt.ylabel("Not sure what these units are yet")
plt.title("Gaussian Smooth Output (sA-sB)/(sA+sB)")

In [ ]:
orbitalf, orbitalamp = LombScargle(time,output).autopower()

In [ ]:
plt.plot(orbitalf, orbitalamp)
plt.xlabel("Frequency(cycles/day)")
plt.ylabel("LombScargle")
plt.xlim(0., 0.1)
plt.ylim(0., 0.005)
#zoom in around f0
#output should show nothing there if well tuned

In [ ]:
# look at width of line in smoothed data
foo = np.zeros_like(time)
foo[5000] = 1.
bar = scipy.ndimage.filters.gaussian_filter1d(foo, 30) # magic
I = (quarter < 100) #error unless at 0?
q,y= LombScargle(time, flux_zeromean * bar).autopower()
plt.title("Zoomed-In Section")
plt.plot(q,y)
plt.xlabel("Frequency(cycles/day)")
plt.ylabel("LombScargle")
plt.xlim(14.1875-0.5, 14.1875+0.5)
plt.axvline(fA)
plt.axvline(fB)
plt.grid()

In [ ]:
plt.plot(time,(sAreal ** 2 + sAimag ** 2))

In [ ]:
plt.plot(time,sBreal)

In [ ]:
plt.plot(time,np.real(rA))

In [ ]:
plt.plot(time,np.real(rB))

In [ ]:
plt.plot(time,np.imag(rA))

In [ ]:
plt.plot(time,np.imag(rB))

In [ ]:
plt.plot(time,sAimag)

In [ ]:
plt.plot(time,sBimag)